Detección de caras con webcam

In [1]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
import math

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [55]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

    # Face detection and eye model setup
imodoF = 1
imodoE = 1

debug = 0

#Set camera resolution
cap.set(3,640);
cap.set(4,480);
sombrero = cv2.imread("gorro-mexicano.png")
bigote = cv2.imread("bigote-mejicano.png")

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            # cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, "Surprise", (10,50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 0))
            #------

            
            center_nose = (shape[30][0], shape[30][1])
            left_nose = (shape[48][0], shape[48][1])
            right_nose = (shape[54][0], shape[54][1])
            nose_width = int(math.hypot(left_nose[0] - right_nose[0],
                            left_nose[1] - right_nose[1]) * 1.7)
            
            nose_height = int(nose_width * 0.45)

            # Nueva posicion de la nariz
            top_left = (int(center_nose[0] - nose_width / 2),
                                int(center_nose[1]))
            
            #Reajuste de la imagen definiendo su posicion dentro del frame del video
            bigote_width = int(nose_width*2)
            bigote_height = int(nose_height*0.5)
            bigote_resized = cv2.resize(bigote, (nose_width, nose_height))
            bigote_gray = cv2.cvtColor(bigote_resized, cv2.COLOR_BGR2GRAY)
            _, nose_mask = cv2.threshold(bigote_gray, 25, 255, cv2.THRESH_BINARY_INV)
            bigote_area = frame[top_left[1]: top_left[1] + nose_height,
                        top_left[0]: top_left[0] + nose_width]
            bigote_area_no_img = cv2.bitwise_and(bigote_area, bigote_area, mask=nose_mask)
            final_nose = cv2.add(bigote_area_no_img, bigote_resized)            
            frame[top_left[1]: top_left[1] + nose_height,
                        top_left[0]: top_left[0] + nose_width] = final_nose

            #---------
            
            hat_size = 3
            proportion = hat_size * 100
            #------ç
            top_head = (shape[31][0], shape[31][1])
            center_head = (shape[27][0], shape[27][1])
            left_head = (shape[17][0], shape[17][1])
            right_head = (shape[26][0], shape[26][1])
            head_width = int(math.hypot(left_head[0] - right_head[0],
                            left_head[1] - right_head[1]) * hat_size)
            
            
            head_height = int(head_width * 0.75)
            # print(left_head)
            
            # Nueva posicion de la nariz
            top_left = (int(center_head[0] - head_width / 2),
                                int(center_head[1] - head_height + 20 ))
            print( int((center_head[1] - head_height )/ 2.5))
            
            #Reajuste de la imagen definiendo su posicion dentro del frame del video
            sombrero_resized = cv2.resize(sombrero, (head_width, head_height))
            sombrero_gray = cv2.cvtColor(sombrero_resized, cv2.COLOR_BGR2GRAY)
            _, head_mask = cv2.threshold(sombrero_gray, 25, 255, cv2.THRESH_BINARY_INV)
            sombrero_area = frame[top_left[1]: top_left[1] + head_height,
                        top_left[0]: top_left[0] + head_width]
            try:
                sombrero_area_no_nose = cv2.bitwise_and(sombrero_area, sombrero_area, mask=head_mask)
                final_head = cv2.add(sombrero_area_no_nose, sombrero_resized)            
                frame[top_left[1]: top_left[1] + head_height,
                        top_left[0]: top_left[0] + head_width] = final_head
            except:
                print("")
            
            #----
            #-----------
            #---------------
            #----------------------
           


            # draws eyes and mask if available
            # [lex, ley, rex, rey] = eyes
            # if lex > -1:
            #     # Show detected facial elements
            #     if imodoF > 0:
            #         for (x, y) in shape:
            #             cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
            #             # print(x, y)

                  


                # Normalize and show
                # color channels
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGR)


    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    # Face detector change
    elif tec & 0xFF == ord('d'):
        imodoF = imodoF + 1
        if imodoF >= len(FDet.FaceDetectors):
            imodoF = 0
    #Eye detector change
    elif tec & 0xFF == ord('e'):
        imodoE = imodoE + 1
        if imodoE >= len(FDet.EyeDetectors):
            imodoE = 0

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0
19
20
20
21
22
23
20
20
19
16
16
23
28
31
32
32
30
29
29
30
31
32
31
33
33
36
40
42
42
43
43
43
42
36
32
28
24
24
22
22
22
20
21
21
21
22
22
22
24
28
36
34
38
39
39
26
22
20
21
22
23
24
23
23
22
22
22
22
21
22
22
22
21
21
23
23
23
22
23
22
23
23
24
25
